In [32]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
data=pd.read_csv('uci-news-aggregator.csv')

In [4]:
data.shape

(422419, 8)

In [62]:
data=data.iloc[:1000,:]

In [63]:
data.shape

(1000, 8)

In [64]:
data.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,0,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,0,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,0,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,0,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,0,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [65]:
raw=[]
for text in data.TITLE:
    raw.append(text)

In [66]:
import re
import string
raw1=[]
for text in raw:
    text=text.lower()
    for z in text:
        if  z in string.punctuation:
            text=text.replace(z,'')
    raw1.append(text)

In [67]:
raw1

['fed official says weak data caused by weather should not slow taper',
 'feds charles plosser sees high bar for change in pace of tapering',
 'us open stocks fall after fed official hints at accelerated tapering',
 'fed risks falling behind the curve charles plosser says',
 'feds plosser nasty weather has curbed job growth',
 'plosser fed may have to accelerate tapering pace',
 'feds plosser taper pace may be too slow',
 'feds plosser expects us unemployment to fall to 62 by the end of 2014',
 'us jobs growth last month hit by weatherfed president charles plosser',
 'ecb unlikely to end sterilisation of smp purchases  traders',
 'ecb unlikely to end sterilization of smp purchases traders',
 'eus halfbaked bank union could work',
 'europe reaches crunch point on banking union',
 'ecb focusstronger euro drowns out ecbs message to keep rates low for  ',
 'eu aims for deal on tackling failing banks',
 'forex  pound drops to onemonth lows against euro',
 'noyer says strong euro creates unw

In [68]:
#2nd approach
def clean_text(text):
    text=re.sub("[0-9]+","/DIG/",text)
    
    return text


In [69]:
raw2=[clean_text(text) for text in raw1]

In [70]:
vectorizer=TfidfVectorizer(analyzer='word',stop_words='english',ngram_range=(1,2))

In [71]:
input_final=vectorizer.fit_transform(raw2)

In [72]:
input_final.shape

(1000, 5075)

In [73]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['CATEGORY']=le.fit_transform(data['CATEGORY'])
cat_data=data['CATEGORY']
cat_data=pd.DataFrame(cat_data)

In [74]:
vectorizer_data=pd.DataFrame(input_final.todense(),columns=vectorizer.get_feature_names())

In [75]:
final_data=pd.concat([vectorizer_data,cat_data],axis=1)

In [80]:
x=final_data.iloc[:,:-1].values
y=final_data.iloc[:,-1].values

In [81]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.3,random_state=20)

In [82]:
from sklearn.svm import SVC
svc=SVC()
svc.fit(xtrain,ytrain)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [83]:
ypred=svc.predict(xtest)

In [86]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(ytest,ypred)

0.96

In [88]:
classification_report(ytest,ypred)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


'              precision    recall  f1-score   support\n\n           0       0.96      1.00      0.98       288\n           1       0.00      0.00      0.00        12\n\n    accuracy                           0.96       300\n   macro avg       0.48      0.50      0.49       300\nweighted avg       0.92      0.96      0.94       300\n'

In [90]:
from sklearn.model_selection import GridSearchCV

In [92]:
params={'kernel':['rbf','poly'],'C':[0.1,1,10,100],'gamma':[0.01,0.1,1,10]}

In [93]:
model=GridSearchCV(SVC(),params)

In [94]:
model.fit(xtrain,ytrain)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='auto_deprecated', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [0.01, 0.1, 1, 10],
                         'kernel': ['rbf', 'poly']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [96]:
model.best_estimator_

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [97]:
ypred=model.predict(xtest)

In [98]:
accuracy_score(ytest,ypred)

0.99

In [102]:
import pickle
import os 
os.chdir(r'F:\mukulml\KAGGLE\uci-news-aggregator.csv')
filename='model.pkl'
with open(filename,'wb') as file:
    pickle.dump(model,file)

In [103]:
with open(filename,'rb') as file:
    classifier=pickle.load(file)

In [108]:
input_line='Fed official says weak data caused by weather, should not slow taper'
def input_text(text):
    text=text.lower()
    for z in text:
        if  z in string.punctuation:
            text=text.replace(z,'')
    text=re.sub("[0-9]+","/DIG/",text)
    
    
    return text
    

In [111]:
input_li=input_text(input_line)
input_li

'fed official says weak data caused by weather should not slow taper'

In [122]:


input_vector=vectorizer.transform([input_li]).todense()

In [124]:
model.predict(input_vector)

array([0], dtype=int64)

In [129]:
input_lin="Galaxy S5 goes on sale in Korea in spite of Samsung's wishes"
def input_text(text):
    text=text.lower()
    for z in text:
        if  z in string.punctuation:
            text=text.replace(z,'')
    text=re.sub("[0-9]+","/DIG/",text)
    
    
    return text
input_li=input_text(input_lin)
input_li


'galaxy s/DIG/ goes on sale in korea in spite of samsungs wishes'